# Forest Cover Classification

### **By Vinesh Raghvani**

In this project, I will be using deep learning to forecast forest cover type (the most prevalent form of tree cover) using just cartographic data. The actual forest cover type for a specific 30 × 30 meter cell was identified using US Forest Service (USFS) Region 2 Resource Information System data. The cover types are as follows:

1. Spruce/Fir
1. Lodgepole Pine
1. Ponderosa Pine
1. Cottonwood/Willow
1. Aspen
1. Douglas-fir
1. Krummholz

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# Going to turn off the GPU usage for tensorflow
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Defining a function to sort and prepare the data for usage 

In [3]:
def prep_data(raw_df):
    """
    Going to process the data for ML/DL algorithms.
    - Separate features and target variables
    - Divide into training and testing dataset
    - Normalize numerical data

    Parameters: 'raw_df' - It is a CSV file that contains the data needed
    Output: X_train, X_test, Y_train, Y_test
    """

    data = raw_df.values
    features, Y = data[:, :-1], data[:, -1]

    # Dividing into training and testing set
    features_train, features_test, Y_train, Y_test = train_test_split(features, Y, test_size=0.20, random_state=42, stratify=Y)

    # Normalize Data
    normalizer = StandardScaler()
    X_train = normalizer.fit_transform(features_train)
    X_test = normalizer.fit_transform(features_test)

    return X_test, X_train, Y_train, Y_test

## Defining a Build Model function 

In [4]:
def create_neural_network(input_features):
    """
    Construct and compile a neural network model.

    Parameters: 'input_features' - Number of input features
    output: Compiled neural network model.
    """
    model = keras.Sequential(name='Neural_Network_Classifier')
    model.add(layers.Dense(units=64, input_shape=(input_features,), activation='relu', name='Input_Layer'))
    #model.add(layers.Dropout(rate=0.3, name='Dropout_1'))
    model.add(layers.Dense(units=32, activation='relu', name='Hidden_Layer_1'))
    #model.add(layers.Dropout(rate=0.3, name='Dropout_2'))
    model.add(layers.Dense(units=8, activation='softmax', name='Output_Layer'))
    model.compile(optimizer=keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'], name='Model_Compilation')
    return model
